In [2]:

import os
from os.path import join
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import torch
import pickle as pkl

In [1]:
savedir = "/n/holylfs06/LABS/kempner_fellow_binxuwang/Users/binxuwang/DL_Projects/DiT/results/045-RAVEN10_abstract-uncond-DiT_S_1_20240311-1256/inpaint_pilot"

In [53]:
stats_df = pd.read_csv(join(savedir, "inpaint_rule_stats.csv"), index_col=0)

In [54]:
stats_df['rule_uniq'] = stats_df['rule_uniq'].apply(lambda x: [int(float(num)) for num in x.strip('[]').split()])
stats_df['rule_cnts'] = stats_df['rule_cnts'].apply(lambda x: [int(float(num)) for num in x.strip('[]').split()])
stats_df['idxs'] = stats_df['idxs'].apply(lambda x: eval(x))

In [55]:
stats_df.shape # (6000 = 40 * 3 pos * 50 rep, 4)

(6000, 8)

In [56]:
stats_df.head()

,iclass,idxs,x_pos,y_pos,consistent_r3,batch_size,rule_uniq,rule_cnts
0,0,"[8513, 5917, 4432]",0,2,78,100,"[0, 26, 31, 6, 21]","[78, 7, 2, 2, 1]"
1,0,"[8513, 5917, 4432]",1,2,0,100,"[25, 35, 16, 12, 26, 6, 21, 36]","[33, 9, 6, 4, 3, 3, 2, 1]"
2,0,"[8513, 5917, 4432]",2,2,57,100,"[0, 12, 30, 16, 5, 29]","[57, 14, 11, 4, 3, 1]"
3,0,"[4114, 7145, 679]",0,2,0,100,"[24, 25, 6, 36, 39, 38]","[14, 6, 4, 3, 1, 1]"
4,0,"[4114, 7145, 679]",1,2,40,100,"[0, 5, 34, 35, 26, 15, 21]","[40, 23, 9, 8, 8, 5, 4]"


In [23]:
print(stats_df.loc[0,"iclass"]) # rule class 
print(stats_df.loc[0,"idxs"]) # index 

0
[8513, 5917, 4432]


In [24]:
stats_df.consistent_r3.mean()

26.145666666666667

In [25]:
stats_df.groupby(["iclass"]).consistent_r3.mean()

iclass
0     38.753333
1      7.953333
2     56.346667
3     35.380000
4     43.800000
5     20.746667
6     31.366667
7      2.753333
8      1.180000
9      5.626667
10    31.066667
11    51.566667
12    38.226667
13    33.373333
14    25.413333
15    14.160000
16     9.213333
17     0.313333
18     0.033333
19     1.620000
20    10.326667
21    50.646667
22    39.140000
23    28.453333
24    22.133333
25    15.560000
26    23.420000
27     0.273333
28     0.106667
29     1.720000
30    62.246667
31    57.213333
32    60.900000
33    72.513333
34    13.113333
35    46.886667
36    38.166667
37     0.233333
38    32.733333
39    21.146667
Name: consistent_r3, dtype: float64

In [26]:
# splitted by x_pos and rule class
reshaped_df = pd.pivot_table(stats_df, values='consistent_r3', index='iclass', columns='x_pos', aggfunc=np.mean)
reshaped_df

/tmp/ipykernel_3143586/1052601614.py:2: FutureWarning: The provided callable <function mean at 0x14db9e6f2e60> is currently using DataFrameGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass 'mean' instead.
  reshaped_df = pd.pivot_table(stats_df, values='consistent_r3', index='iclass', columns='x_pos', aggfunc=np.mean)


x_pos,0,1,2
iclass,,,
0,42.12,36.36,37.78
1,8.36,7.14,8.36
2,56.22,55.46,57.36
3,38.28,39.42,28.44
4,39.46,43.40,48.54
5,19.06,24.42,18.76
6,29.52,29.70,34.88
7,2.32,2.06,3.88
8,0.78,0.82,1.94


### Wrong inference

In [65]:
target_class = 0
rule_cnter = np.zeros(40, dtype=int)
for idx, row in stats_df[stats_df.iclass==target_class].iterrows():
    rule_cnter[row.rule_uniq] += row.rule_cnts
rule_cnter

array([5813,    0,    0,    0,    0,  645,  509,    0,    0,    0,  135,
        126,  110,  221,   60,  510,  472,    2,    0,   41,  136,  163,
         96,  169,   48,  695,  545,    2,    1,   25,  133,   60,   89,
         95,   45,  300,  334,   27,   37,   30])

In [66]:
!cp /n/holylfs06/LABS/kempner_fellow_binxuwang/Users/binxuwang/DL_Projects/DiT/results/045-RAVEN10_abstract-uncond-DiT_S_1_20240311-1256/inpaint_pilot/inpaint_rule_stats.csv inpaint_rule_stats.csv